## 1. Configuração e Carregamento

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler # <-- Importante: para normalizar os scores

# --- Configurações ---
K = 3 # O número de recomendações final
K_SVD = 2 # O número de fatores latentes (igual ao 04_CF)

# Pesos do Híbrido:
# Daremos mais peso ao CF (ML Puro) por ser mais "inteligente"
W_CF = 0.6  # 60%
W_CB = 0.4  # 40%

print("Bibliotecas carregadas.")

Bibliotecas carregadas.


## 2. Carregar Todos os Dados

In [5]:
try:
    df_provincias_feat = pd.read_csv('../data/provincia_features_MVP.csv', index_col='Provincia')
    df_users_feat = pd.read_csv('../data/user_features_MVP.csv', index_col='user_id')
    df_interactions = pd.read_csv('../data/user_interactions_MVP.csv')
    print("Ficheiros MVP carregados com sucesso.")
except FileNotFoundError:
    print("ERRO: Ficheiros MVP não encontrados na pasta /data/!")

Ficheiros MVP carregados com sucesso.


## 3. Recriar o Motor 1: Content-Based (CB)

In [6]:
print("\n--- A executar: Motor 1 (Content-Based) ---")
# 1. Preparar features (igual ao 03_CB)
features_das_provincias = df_provincias_feat[['impacto_ambiental', 'feat_cultura', 'feat_praia', 'feat_aventura', 'feat_natureza']]
features_dos_utilizadores = df_users_feat[['pref_sustentavel', 'pref_cultura', 'pref_praia', 'pref_aventura', 'pref_natureza']]
features_dos_utilizadores.columns = features_das_provincias.columns # Alinhar nomes

# 2. Calcular scores CB
matriz_similaridade = cosine_similarity(
    features_dos_utilizadores.values, 
    features_das_provincias.values
)
df_scores_cb = pd.DataFrame(
    matriz_similaridade,
    index=features_dos_utilizadores.index,
    columns=features_das_provincias.index
)

# 3. Normalizar os scores CB (0 a 1)
# O Cosine Similarity já está numa escala de 0-1, mas o MinMaxScaler garante.
scaler_cb = MinMaxScaler()
scores_cb_normalized = scaler_cb.fit_transform(df_scores_cb)
df_scores_cb_norm = pd.DataFrame(scores_cb_normalized, index=df_scores_cb.index, columns=df_scores_cb.columns)

print("Scores do Content-Based (CB) calculados e normalizados.")
print(df_scores_cb_norm.round(2).head())


--- A executar: Motor 1 (Content-Based) ---
Scores do Content-Based (CB) calculados e normalizados.
Provincia  Luanda  Benguela  Huíla  Namibe  Malanje  Cabinda  Huambo  \
user_id                                                                
1            1.00      1.00   0.00    0.00     0.00     0.00    0.00   
2            0.00      0.00   0.85    1.00     0.74     0.12    0.53   
3            0.55      0.32   1.00    0.57     1.00     1.00    1.00   

Provincia  Cuando Cubango  
user_id                    
1                    0.00  
2                    1.00  
3                    0.84  


## 4. Recriar o Motor 2: Collaborative Filtering (CF / SVD)

In [7]:
print("\n--- A executar: Motor 2 (Collaborative Filtering) ---")
# 1. Criar a Matriz (igual ao 04_CF)
df_matrix = df_interactions.pivot(
    index='user_id',
    columns='provincia_visitada',
    values='rating'
).fillna(0)
matrix_esparsa = csr_matrix(df_matrix.values)

# 2. Treinar o SVD
model = TruncatedSVD(n_components=K_SVD, random_state=42)
model.fit(matrix_esparsa)

# 3. Gerar Previsões (matriz reconstruída)
factores_user = model.transform(matrix_esparsa)
factores_item = model.components_
matriz_reconstruida_scores = np.dot(factores_user, factores_item)

df_previsoes_svd = pd.DataFrame(
    matriz_reconstruida_scores, 
    index=df_matrix.index, 
    columns=df_matrix.columns
)

# 4. Normalizar os scores CF (CRUCIAL!)
# Os scores do SVD (ex: 0.49, 5.0, -0.0) não estão na escala 0-1.
scaler_cf = MinMaxScaler()
scores_cf_normalized = scaler_cf.fit_transform(df_previsoes_svd)
df_scores_cf_norm = pd.DataFrame(scores_cf_normalized, index=df_previsoes_svd.index, columns=df_previsoes_svd.columns)

print("Scores do Collaborative Filtering (CF) calculados e normalizados.")
print(df_scores_cf_norm.round(2).head())


--- A executar: Motor 2 (Collaborative Filtering) ---
Scores do Collaborative Filtering (CF) calculados e normalizados.
provincia_visitada  Benguela  Cuando Cubango  Huambo  Huíla  Luanda  Malanje
user_id                                                                     
1                       1.00             0.0     0.0    1.0    1.00      0.0
2                       0.03             1.0     1.0    0.0    0.03      1.0
3                       0.00             0.8     0.8    0.0    0.00      0.8


## 5. O Cérebro Híbrido: Combinar os Scores

In [8]:
print("\n--- A executar: O Cérebro Híbrido ---")

# Criar um DataFrame final alinhado
# (Usamos .reindex para garantir que ambos têm as mesmas colunas/linhas)
users_finais = df_scores_cb_norm.index
itens_finais = df_provincias_feat.index # Todas as 8 províncias

# Alinhar CB (já tem todos os users e itens)
cb_final = df_scores_cb_norm.reindex(index=users_finais, columns=itens_finais, fill_value=0)

# Alinhar CF (só tem dados para 6 províncias e 3 users)
cf_final = df_scores_cf_norm.reindex(index=users_finais, columns=itens_finais, fill_value=0)

# A FÓRMULA HÍBRIDA
df_scores_hibridos = (cb_final * W_CB) + (cf_final * W_CF)

print("Scores Híbridos calculados (CB * 40% + CF * 60%).")
print(df_scores_hibridos.round(2))


--- A executar: O Cérebro Híbrido ---
Scores Híbridos calculados (CB * 40% + CF * 60%).
Provincia  Luanda  Benguela  Huíla  Namibe  Malanje  Cabinda  Huambo  \
user_id                                                                
1            1.00      1.00   0.60    0.00     0.00     0.00    0.00   
2            0.02      0.02   0.34    0.40     0.90     0.05    0.81   
3            0.22      0.13   0.40    0.23     0.88     0.40    0.88   

Provincia  Cuando Cubango  
user_id                    
1                    0.00  
2                    1.00  
3                    0.81  


## 6. Gerar Recomendações Finais (Com Lógica "Cold Start")

In [9]:
def get_hybrid_recommendations(user_id, k, df_hibrido, df_cb_norm):
    """
    Gera recomendações híbridas.
    Resolve o "Cold Start" (novo utilizador) usando apenas CB.
    """
    if user_id not in df_hibrido.index:
        # --- LÓGICA DE "COLD START" (NOVO UTILIZADOR) ---
        # print(f"User {user_id} é novo. A usar 100% Content-Based.")
        user_scores = df_cb_norm.loc[user_id]
    else:
        # --- LÓGICA HÍBRIDA (UTILIZADOR EXISTENTE) ---
        user_scores = df_hibrido.loc[user_id]
        
    # Ordenar do maior para o menor e obter o Top-K
    recs = user_scores.sort_values(ascending=False).index.tolist()
    
    # Remover itens que o utilizador JÁ AVALIOU (Opcional, mas boa prática)
    # Vamos implementar isto para a defesa final:
    # avaliados = []
    # if user_id in df_matrix.index: # Verificar se o user tem historial
    #    avaliados = df_matrix.loc[user_id][df_matrix.loc[user_id] > 0].index.tolist()
        
    #recs_sem_avaliados = [item for item in recs if item not in avaliados]
    
    return recs[:k]

# Gerar recomendações finais para todos
recomendacoes_finais = {}
for user_id in df_users_feat.index: # Iterar por TODOS os utilizadores (incluindo novos)
    recomendacoes_finais[user_id] = get_hybrid_recommendations(user_id, K, df_scores_hibridos, df_scores_cb_norm)

print("\n--- Recomendações Finais (Híbrido) ---")
for user, recs in recomendacoes_finais.items():
    print(f"User {user}: {recs}")


--- Recomendações Finais (Híbrido) ---
User 1: ['Luanda', 'Benguela', 'Huíla']
User 2: ['Cuando Cubango', 'Malanje', 'Huambo']
User 3: ['Huambo', 'Malanje', 'Cuando Cubango']


In [10]:
print("\n--- A calcular métricas finais para o Híbrido ---")

# 1. Obter o "Ground Truth" (O que os utilizadores REALMENTE gostaram - ratings >= 4)
ground_truth = {}
for user in df_interactions['user_id'].unique():
    user_likes = df_interactions[
        (df_interactions['user_id'] == user) & (df_interactions['rating'] >= 4)
    ]['provincia_visitada'].tolist()
    ground_truth[user] = user_likes

# 2. Corrigir a lógica de recomendação (como discutido)
recomendacoes_finais_corrigidas = {}
for user_id in df_users_feat.index:
    
    if user_id not in df_scores_hibridos.index:
        # Lógica de "Cold Start" (não temos o User 3 no CF)
        # Vamos usar o User 3 do CB como exemplo
        if user_id == 3:
            user_scores = df_scores_cb_norm.loc[user_id]
        else:
             user_scores = df_scores_cb_norm.loc[user_id] # Default
    else:
        # Lógica Híbrida
        user_scores = df_scores_hibridos.loc[user_id]
        
    recs = user_scores.sort_values(ascending=False).index.tolist()
    recomendacoes_finais_corrigidas[user_id] = recs[:K] # Top-K directo

print("Recomendações Finais (Corrigidas):")
print(recomendacoes_finais_corrigidas)
    
# 3. Calcular Métricas de Relevância (Precisão, Recall)
lista_precision = []
lista_recall = []

for user, likes in ground_truth.items():
    if not likes or user not in recomendacoes_finais_corrigidas:
        continue 

    recs_do_user = recomendacoes_finais_corrigidas[user]
    hits = len(set(recs_do_user) & set(likes))
    
    precision_k = hits / K
    lista_precision.append(precision_k)
    
    recall_k = hits / len(likes)
    lista_recall.append(recall_k)

print("\n--- Métricas de Relevância (Híbrido) ---")
precision_final = np.mean(lista_precision)
recall_final = np.mean(lista_recall)
print(f"Precisão@ {K} (Média): {precision_final * 100:.2f}%")
print(f"Recall@ {K} (Média):   {recall_final * 100:.2f}%")

# 4. Calcular Métricas de Plataforma e Impacto
mainstream_list = df_provincias_feat.sort_values(by='popularidade', ascending=False).index[:2].tolist()
sustainable_list = df_provincias_feat[df_provincias_feat['impacto_ambiental'] >= 0.7].index.tolist()

lista_diversidade = []
lista_sustentabilidade = []

for user, recs in recomendacoes_finais_corrigidas.items():
    emergentes_recomendados = len(set(recs) - set(mainstream_list))
    sustentaveis_recomendados = len(set(recs) & set(sustainable_list))
    
    lista_diversidade.append(emergentes_recomendados / K)
    lista_sustentabilidade.append(sustentaveis_recomendados / K)

print("\n--- Métricas de Plataforma e Impacto (Híbrido) ---")
diversidade_final = np.mean(lista_diversidade)
sustentabilidade_final = np.mean(lista_sustentabilidade)
print(f"Diversidade@ {K} (% Emergentes):   {diversidade_final * 100:.2f}%")
print(f"Sustentabilidade@ {K} (% Sustentáveis): {sustentabilidade_final * 100:.2f}%")


--- A calcular métricas finais para o Híbrido ---
Recomendações Finais (Corrigidas):
{1: ['Luanda', 'Benguela', 'Huíla'], 2: ['Cuando Cubango', 'Malanje', 'Huambo'], 3: ['Huambo', 'Malanje', 'Cuando Cubango']}

--- Métricas de Relevância (Híbrido) ---
Precisão@ 3 (Média): 66.67%
Recall@ 3 (Média):   100.00%

--- Métricas de Plataforma e Impacto (Híbrido) ---
Diversidade@ 3 (% Emergentes):   77.78%
Sustentabilidade@ 3 (% Sustentáveis): 55.56%


## 6.1 Serielização

In [11]:
# 1. Definir o diretório de destino
OUTPUT_DIR = 'models' 
os.makedirs(OUTPUT_DIR, exist_ok=True) # Garantir que a pasta existe

print(f"\n--- A guardar componentes do modelo em {OUTPUT_DIR}/ ---")

# 2. Guardar o modelo SVD treinado (Collaborative Filtering)
# Variável: model (do TruncatedSVD)
joblib.dump(model, os.path.join(OUTPUT_DIR, 'svd_model.pkl'))
print("svd_model.pkl (Modelo CF) guardado.")

# 3. Guardar o scaler treinado para o CF (importante para a normalização)
# Variável: scaler_cf (do MinMaxScaler)
joblib.dump(scaler_cf, os.path.join(OUTPUT_DIR, 'cf_scaler.pkl'))
print("cf_scaler.pkl (Scaler CF) guardado.")

# 4. Guardar as features das Províncias (Content-Based)
# Variável: features_das_provincias (do Passo 3)
joblib.dump(features_das_provincias, os.path.join(OUTPUT_DIR, 'cb_provincias_features.pkl'))
print("cb_provincias_features.pkl (Features CB) guardado.")

# 5. Guardar os nomes das features do Utilizador (Para o CB saber a ordem)
# Variável: features_dos_utilizadores.columns (do Passo 3)
joblib.dump(features_dos_utilizadores.columns, os.path.join(OUTPUT_DIR, 'cb_user_feature_names.pkl'))
print("cb_user_feature_names.pkl (Nomes das Features) guardado.")

# 6. Guardar a matriz de interações completa (Necessário para a lógica Híbrida/CF)
# Variável: df_matrix (do Passo 4)
joblib.dump(df_matrix, os.path.join(OUTPUT_DIR, 'interaction_matrix.pkl'))
print("interaction_matrix.pkl (Matriz de Interações) guardado.")

print("\n--- Serialização Completa. Pode avançar para o Deploy. ---")


--- A guardar componentes do modelo em models/ ---
svd_model.pkl (Modelo CF) guardado.
cf_scaler.pkl (Scaler CF) guardado.
cb_provincias_features.pkl (Features CB) guardado.
cb_user_feature_names.pkl (Nomes das Features) guardado.
interaction_matrix.pkl (Matriz de Interações) guardado.

--- Serialização Completa. Pode avançar para o Deploy. ---


## 7. Conclusões Finais: A Tabela de Comparação da Plataforma

O nosso "sprint" de 4 notebooks pode ser resumido nesta tabela de comparação. Ela prova como cada modelo evoluiu, resolvendo as fraquezas do anterior.

### Comparação Final de Métricas (K=3)

| Métrica | 02_Baseline (Popularidade) | 03_Content-Based (Regras) | 04_CF (ML Puro / SVD) | 05_Hybrid (Final) |
| :--- | :---: | :---: | :---: | :---: |
| **`Precisão@3`** (Relevância) | 22.22% | 66.67% | 66.67% | **66.67%** |
| **`Recall@3`** (Relevância) | 33.33% | 100.00% | 100.00% | **100.00%** |
| **`Diversidade@3`** (Plataforma) | 33.33% | 77.78% | 77.78% | **77.78%** |
| **`Sustentabilidade@3`** (Impacto)| 33.33% | **66.67%** | 55.56% | **55.56%** |



### Análise da "História"

1.  **Baseline vs. Modelos Inteligentes:** A tabela mostra um salto claro. Todos os nossos modelos inteligentes (CB, CF, Híbrido) **esmagaram** o Baseline em todas as métricas, provando que a personalização funciona. O `Recall@3` de 100% mostra que fomos capazes de recomendar *todos* os itens relevantes para os nossos utilizadores.

2.  **A Fraqueza do "ML Puro" (CF):** O nosso modelo `04_CF (SVD)` foi poderoso, mas "cego". Ele foi o *pior* na métrica de `Sustentabilidade@3` (55.56%), porque ele só se preocupa com padrões de ratings, ignorando as metas de impacto.

3.  **A "Vitória" do Híbrido:** O nosso `05_Hybrid_Model` final representa a solução de plataforma ideal.
    * Ele mantém a **performance de relevância** perfeita (100% Recall) e **diversidade** (77.78%) dos outros modelos.
    * Ele equilibra a `Sustentabilidade@3`. O "boost" de 40% do CB puxou a métrica de 33% (Baseline) para 55.56%. Embora não seja tão alto quanto o CB *puro* (66.67%), isto é um *trade-off* (troca) intencional: sacrificamos ligeiramente o impacto para ganhar o poder de "descoberta" (serendipidade) do SVD.
    * Mais importante, o nosso Híbrido é **à prova de falhas**, usando 100% do CB para resolver o problema do "Cold Start", algo que o SVD sozinho não pode fazer.

**Resultado Final:**
Construímos uma arquitectura de ML defensável que equilibra de forma inteligente a **Satisfação do Utilizador** (aprendizagem do CF) com as **Metas da Plataforma** (regras do CB).